In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, MobileNetV3Small

In [7]:
MobileNetV3Small().save('mobilenetv3.h5')
VGG16().save('vgg16.h5')


# Mobilenetv3 Unet - Mobile semantic segmentation neural network

In [5]:
def build_mobilenetv3_unet(input_shape):
    """ input """
    inputs = Input(input_shape)
    mobilenetv3small = MobileNetV3Small(
    include_top=False,
    weights='imagenet',
    input_tensor=inputs
    )
    return mobilenetv3small

In [3]:
input_shape=(320, 320, 3)
model = build_mobilenetv3_unet(input_shape)

In [4]:
model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 320, 320, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 160, 160, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                   